In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

In [2]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [3]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [5]:
client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain AI to me like I'm a kid.")

print(response.text)

Okay, imagine you have a really smart puppy.  It doesn't know anything at first, right?  But you can teach it tricks!

AI is kind of like that puppy, but it's a computer program instead of a furry friend.  We can teach it things by showing it lots and lots of examples.

*   **Imagine you want to teach the puppy to recognize a ball.** You show it hundreds of different balls: big ones, small ones, red ones, blue ones, even flat ones!  After seeing enough, the puppy learns what all those balls have in common and can recognize a new ball, even if it's different from the ones it saw before.

*   **AI is the same!**  If we want an AI to recognize cats in pictures, we show it millions of pictures of cats. It looks at all the whiskers, the pointy ears, the furry bodies, and learns what makes a cat a cat.  Then, when we show it a new picture, it can say, "Hey, that's a cat!"

So, AI is like a super-smart puppy (or a super-smart student) that learns from lots of examples to do things like:

*   

In [6]:
Markdown(response.text)

Okay, imagine you have a really smart puppy.  It doesn't know anything at first, right?  But you can teach it tricks!

AI is kind of like that puppy, but it's a computer program instead of a furry friend.  We can teach it things by showing it lots and lots of examples.

*   **Imagine you want to teach the puppy to recognize a ball.** You show it hundreds of different balls: big ones, small ones, red ones, blue ones, even flat ones!  After seeing enough, the puppy learns what all those balls have in common and can recognize a new ball, even if it's different from the ones it saw before.

*   **AI is the same!**  If we want an AI to recognize cats in pictures, we show it millions of pictures of cats. It looks at all the whiskers, the pointy ears, the furry bodies, and learns what makes a cat a cat.  Then, when we show it a new picture, it can say, "Hey, that's a cat!"

So, AI is like a super-smart puppy (or a super-smart student) that learns from lots of examples to do things like:

*   **Recognize things:** Like cats, dogs, faces, or even letters in a word.
*   **Understand language:** So it can answer your questions or translate languages.
*   **Play games:** Really, really well, like chess or video games.
*   **Drive cars:**  By learning from lots of driving experiences.

The more we teach AI, the smarter it gets. It's like that puppy learning more and more tricks!  It's still learning, and it's not perfect, but it's getting better all the time!


In [7]:
chat = client.chats.create(model='gemini-2.0-flash', history=[])
response = chat.send_message('Hello! My name is Hemant.')
print(response.text)

Hello Hemant! It's nice to meet you. How can I help you today?



In [8]:
response = chat.send_message('Can you tell me something interesting about black holes?')
Markdown(response.text)

Okay, here's a fascinating fact about black holes that often blows people's minds:

**Black holes aren't actually "holes." They're incredibly dense objects.**

We often think of them as giant cosmic vacuum cleaners that suck everything up, and the "hole" part of the name implies emptiness.  However, black holes are formed when a massive star collapses in on itself.  All of its matter is compressed into an incredibly small space, creating a point of infinite density called a singularity.  It's this extreme density and the resulting intense gravity that warps spacetime and prevents anything, not even light, from escaping beyond the event horizon.

So, while they *act* like a point of no return, they're actually the result of matter being crushed beyond comprehension, not just a literal empty void.

Did you find that interesting? I can tell you more if you'd like!


In [9]:
response = chat.send_message('Do you remember what my name is?')
print(response.text)

Yes, your name is Hemant.



In [10]:
for model in client.models.list():
  print(model.name)

models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
mo

In [11]:
from pprint import pprint

for model in client.models.list():
  if model.name == 'models/gemini-2.0-flash':
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent',
                       'countTokens',
                       'createCachedContent',
                       'batchGenerateContent'],
 'tuned_model_info': {},
 'version': '2.0'}


In [13]:
short_config = types.GenerateContentConfig(max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write 1000 word essay on modern corporate life and stress that comes with it.'
)

Markdown(response.text)

## The Algorithmic Cage: Stress and the Soul in Modern Corporate Life

Modern corporate life, with its gleaming skyscrapers, digital ubiquity, and promises of success, often feels less like a pathway to fulfillment and more like a complex, high-stakes game. Driven by relentless demands for productivity, efficiency, and innovation, the contemporary workplace has become a breeding ground for stress, impacting not only individual well-being but also the very fabric of our social lives. This essay will delve into the multifaceted nature of stress within the modern corporate landscape, exploring its sources, consequences, and the growing urgency to address its pervasive impact on the human spirit.

One of the primary drivers of corporate stress is the unrelenting pressure to perform. The rise of metrics-driven management has transformed employees into data points, constantly evaluated and compared against their peers. Key Performance Indicators (KPIs) have become the measuring stick of success, pushing individuals to prioritize quantifiable results over intrinsic motivation and creativity. This relentless pursuit of targets fosters

In [14]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write short poem on modern corporate life and stress that comes with it.'
)

Markdown(response.text)

Sunrise to screen's pale gleam,
Another day, a waking dream.
Metrics hum, a constant drone,
Another fire to put alone.

Emails flood, a rising tide,
While deadlines loom, with nowhere to hide.
Smile pasted, a practiced art,
Hiding the crack within my heart.

Performance reviews, a judging eye,
Another piece of me to die.
The climb is steep, the air is thin,
When does the victory truly begin?


In [15]:
temp_config = types.GenerateContentConfig(temperature=2.0)

for _ in range(5):
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        config=temp_config,
        contents='generate a color (respond with just one word)'
    )

    if response.text:
        print(response.text)
        print('#############')

Cerulean

#############
Azure

#############
Azure

#############
Cerulean

#############
Azure

#############


In [16]:
temp_config = types.GenerateContentConfig(temperature=0.0)

for _ in range(5):
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        config=temp_config,
        contents='generate a color (respond with just one word)'
    )

    if response.text:
        print(response.text)
        print('#############')

Azure

#############
Azure

#############
Azure

#############
Azure

#############
Azure

#############


In [17]:
temp_config = types.GenerateContentConfig(temperature=0.2)

for _ in range(5):
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        config=temp_config,
        contents='generate a color (respond with just one word)'
    )

    if response.text:
        print(response.text)
        print('#############')

Azure

#############
Azure

#############
Azure

#############
Azure

#############
Azure

#############


In [18]:
model_config = types.GenerateContentConfig(
    temperature=2.0,
    top_p=0.95,
)

prompt = 'Write a story about a cat who quits his corporate job and retires to a peaceful life in the mountains'

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=prompt
)

Markdown(response.text)

Bartholomew "Barty" Whiskers was a creature of habit. For ten years, he'd meticulously smoothed his fur each morning, precisely at 6:15 AM, before battling the rush-hour crawl (kitty-sized bicycle, of course) to get to his Senior Executive Mouse Chaser position at Purrfect Incorporated. He’d delivered stellar Q3 reports, expertly negotiated feather-toy contracts, and climbed the corporate ladder with ruthless ambition. But deep down, Barty felt…empty.

The joy had been gnawed away, leaving only the stale cheese crackers of stress and the echoing loneliness of a life chasing after something that felt utterly meaningless. He stared at his reflection one day, seeing a greying muzzle, weariness in his emerald eyes. "There has to be more to life than chasing tiny rodents," he muttered, pawing at the mahogany desk that symbolized his success.

That night, Barty wrote his resignation letter with trembling claws. He quoted Thoreau, a long-time favorite, much to the chagrin of HR (they had a strict “No Emersonian philosophy in exit interviews” policy). He then cleaned out his office, meticulously removing every trace of his decade-long imprisonment. As he slipped his cat-shaped laptop into his messenger bag, he knew he was about to trade cubicles for cabins, spreadsheets for sunsets, and stress for…well, hopefully, significantly less stress.

He cashed in his stock options, packed a duffel bag with essentials (mostly yarn and a favorite salmon-flavored dental chew), and boarded a rickety bus bound for the Whisperwind Mountains.

Barty bought a dilapidated cabin overlooking a valley dusted with wildflowers. It needed work – the roof leaked, the chimney coughed smoke, and the resident family of mice had a better-established residency than he did. But Barty loved it. The air was crisp, the sky vast, and the silence, initially deafening, slowly became a balm for his frayed nerves.

His days settled into a peaceful rhythm. He woke to the chirping of birds, stretched languidly, and brewed a cup of catnip tea (much superior to the company-provided blend). He spent hours exploring the trails, chasing butterflies, and napping under the watchful gaze of ancient pines. He learned to identify different types of birdsong, became intimately familiar with the local squirrel gossip, and even befriended a gruff old badger named Agnes who offered sage advice (and delicious berries).

The mice in his cabin, initially hostile, eventually became amiable housemates. He named them Milton, Reginald, and Beatrice, and even shared his tuna-flavored cat food with them (much to the horror of his past self).

He missed some aspects of his old life. The satisfying *thwack* of landing a lucrative deal, the feeling of power in the board meetings. But those feelings were fleeting compared to the profound peace he found in the mountains.

One evening, sitting on his porch, bathed in the golden light of the setting sun, Barty saw a hawk circling overhead. He watched, mesmerized, as it soared effortlessly, free from spreadsheets and stress, utterly content. He felt a kinship, a silent understanding between two creatures finally living on their own terms.

A small, contented purr rumbled in his chest. He reached for his yarn ball, its soft wool comforting in his paws. He hadn't chased a mouse in weeks, and he couldn't be happier. He'd traded the cutthroat corporate jungle for the sweet symphony of the mountains. Barty Whiskers, former Senior Executive Mouse Chaser, was finally, truly, retired. And it felt purrfect.


In [19]:
model_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=1,
    max_output_tokens=5
)

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=zero_shot_prompt
)

print(response.text)

POSITIVE



In [24]:
import enum

class Sentiment(enum.Enum):
    POSITIVE= 'pos'
    NEGETIVE= 'neg'
    NEUTRAL= 'neu'

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ),
    contents=zero_shot_prompt
)

print(response.text)

pos


In [46]:
enum_response = response.parsed
print(enum_response)
print(type(enum_response))

Sentiment.POSITIVE
<enum 'Sentiment'>


In [41]:
from google.genai.types import Part

class CustomerIntent(enum.Enum):
    CANCEL_ORDER = 'cancel_order'
    CHECK_STATUS = 'check_status'
    REQUEST_REFUND = 'request_refund'
    GENERAL_QUERY = 'general_query'
    PRODUCT_INFO = 'product_info'

SYSTEM_PROMPT = (
    "You are an expert intent classifier for a retail company. "
    "Your task is to analyze the user's message and determine the single, primary intent. "
    "You MUST only respond with one of the provided enum values."
)

def classify_intent(user_message):
    contents = [
        # Put the system prompt as a user-role content (server accepts "user")
        types.Content(
            role="user",
            parts=[Part.from_text(text=SYSTEM_PROMPT)]
        ),
        # Actual user message
        types.Content(
            role="user",
            parts=[Part.from_text(text=user_message)]
        ),
    ]


    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=contents, 
            config=types.GenerateContentConfig(
                response_mime_type="text/x.enum",
                response_schema=CustomerIntent
            )
        )
        return response.text.strip()
    
    except Exception as e:
        return f"API Error: {e}"

In [44]:
message_1 = "I need to get my money back because the item arrived broken."
intent_1 = classify_intent(message_1)
print(intent_1)

message_2 = "Can you tell me where my package is right now? I ordered it last week."
intent_2 = classify_intent(message_2)

message_3 = "Does the new X-Pro 500 come in red, and what are the main features?"
intent_3 = classify_intent(message_3)

request_refund


In [45]:
def trigger_workflow(intent: str, message: str):
    """Simulates a backend routing workflow based on the classified intent."""
    
    print(f"\nUser Message: '{message}'")
    print(f"Classified Intent: {intent}")
    
    match intent:
        case CustomerIntent.REQUEST_REFUND.value:
            print("Routing: Initiating Refund Request Workflow.")
        case CustomerIntent.CHECK_STATUS.value:
            print("Routing: Querying Shipping Database for Order Status.")
        case CustomerIntent.CANCEL_ORDER.value:
            print("Routing: Sending Order Cancellation Confirmation.")
        case CustomerIntent.PRODUCT_INFO.value:
            print("Routing: Connecting to Product Knowledge Base.")
        case _:
            # This should not happen if the enum enforcement works
            print("ERROR: Received an invalid intent. Escalating to human review.")


# Run the workflow simulation
trigger_workflow(intent_1, message_1)
trigger_workflow(intent_2, message_2)
trigger_workflow(intent_3, message_3)


User Message: 'I need to get my money back because the item arrived broken.'
Classified Intent: request_refund
Routing: Initiating Refund Request Workflow.

User Message: 'Can you tell me where my package is right now? I ordered it last week.'
Classified Intent: check_status
Routing: Querying Shipping Database for Order Status.

User Message: 'Does the new X-Pro 500 come in red, and what are the main features?'
Classified Intent: product_info
Routing: Connecting to Product Knowledge Base.


In [49]:
few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

ORDER:
"""

customer_order = "Give me a large with half margherita & half burrata salad"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, customer_order])


print(response.text)

```json
{
  "size": "large",
  "type": "half_and_half",
  "ingredients_1": ["tomato sauce", "mozzarella", "basil"],
  "ingredients_2": ["burrata", "salad"]
}
```


In [50]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ),
    contents="Can I have a large margherita pizza?")

print(response.text)

{
  "size": "large",
  "ingredients": ["tomato sauce", "mozzarella cheese", "basil"],
  "type": "margherita"
}


In [51]:
prompt = """When I was 8 years old, my sister was half my age. Now, I
am 34 years old. How old is my sister? Return the answer directly."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

print(response.text)

30



In [52]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? Return the answer directly."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

print(response.text)

48



In [53]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now,
I am 20 years old. How old is my partner? Let's think step by step."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

Markdown(response.text)

Here's how to solve the problem:

1.  **Find the age difference:** When you were 4, your partner was 3 times your age, meaning they were 4 * 3 = 12 years old. The age difference between you and your partner is 12 - 4 = 8 years.

2.  **Calculate the partner's current age:** Since the age difference remains constant, your partner is always 8 years older than you. Now that you are 20, your partner is 20 + 8 = 28 years old.

So the answer is 28


In [54]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

In [55]:
question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

react_config = types.GenerateContentConfig(
    stop_sequences=["\nObservation"],
    system_instruction=model_instructions + example1 + example2,
)


react_chat = client.chats.create(
    model='gemini-2.0-flash',
    config=react_config,
)

resp = react_chat.send_message(question)
print(resp.text)

Thought 1
I need to find the transformers NLP paper, identify the authors, and then determine the youngest author.

Action 1
<search>transformers NLP paper</search>



In [56]:
observation = """Observation 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.
"""
resp = react_chat.send_message(observation)
print(resp.text)

Thought 2
Now I have the authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin. I need to find the youngest of these people. I can search for their ages/birthdates individually.

Action 2
<search>Ashish Vaswani birthdate</search>

